# 03 — Data Cleaning and Merging

**Goal:** Clean each dataset, merge into one master DataFrame, compute clean sheets, and split by position group.

**Inputs:** `../data/raw/*.csv`

**Outputs:** `../data/processed/merged_*.csv` (complete + per position group)

**Next:** `04_feature_engineering.ipynb`

---
**EDA findings driving cleaning decisions (from 02_eda_raw.ipynb):**
- `market_value_in_eur` is heavily right-skewed (median €0.2M) → add `log_market_value` as model target
- `agent_name` (48% missing) and `contract_expiration_date` (36% missing) → dropped
- `height_in_cm` has erroneous values < 100 cm → set to NaN and median-imputed
- 189 players have `position = 'Missing'` → dropped; 49 have missing `date_of_birth` → dropped
- Age range is 15–57; players > 45 are retired with stale data → filtered out
- `highest_market_value_in_eur` only ~8% missing → kept as a feature (peak career value)
- Outliers capped at 99th pct here: goals=78, assists=53, minutes=28528, appearances=376
- `total_red` has median=0 for most players → kept but flagged as low-signal
- `valuation_year` added so notebook 05 can filter by recency (2024 mean spiked to €3.7M)
- `has_appearances` flag added (median appearances=30; players with 0 need distinguishing)

In [ ]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

RAW_PATH    = '../data/raw/'
PROC_PATH   = '../data/processed/'
MERGED_PATH = '../data/processed/merged/'
os.makedirs(PROC_PATH,   exist_ok=True)
os.makedirs(MERGED_PATH, exist_ok=True)

print('Paths OK')
print(f'  raw:    {os.path.abspath(RAW_PATH)}')
print(f'  merged: {os.path.abspath(MERGED_PATH)}')

## 1. Load Datasets

In [3]:
players     = pd.read_csv(RAW_PATH + 'players.csv',           low_memory=False)
appearances = pd.read_csv(RAW_PATH + 'appearances.csv',       low_memory=False)
valuations  = pd.read_csv(RAW_PATH + 'player_valuations.csv', low_memory=False)
club_games  = pd.read_csv(RAW_PATH + 'club_games.csv',        low_memory=False)
lineups     = pd.read_csv(RAW_PATH + 'game_lineups.csv',      low_memory=False)
clubs       = pd.read_csv(RAW_PATH + 'clubs.csv',             low_memory=False)
transfers   = pd.read_csv(RAW_PATH + 'transfers.csv',         low_memory=False)

players['date_of_birth']   = pd.to_datetime(players['date_of_birth'],  errors='coerce')
valuations['date']         = pd.to_datetime(valuations['date'],         errors='coerce')
transfers['transfer_date'] = pd.to_datetime(transfers['transfer_date'], errors='coerce')

print('Loaded:')
print(f'  players:     {len(players):>8,} rows')
print(f'  appearances: {len(appearances):>8,} rows')
print(f'  valuations:  {len(valuations):>8,} rows')
print(f'  club_games:  {len(club_games):>8,} rows')
print(f'  lineups:     {len(lineups):>8,} rows')
print(f'  clubs:       {len(clubs):>8,} rows')
print(f'  transfers:   {len(transfers):>8,} rows')

Loaded:
  players:       34,291 rows
  appearances: 1,722,865 rows
  valuations:   448,965 rows
  club_games:   155,990 rows
  lineups:     2,680,694 rows
  clubs:            451 rows
  transfers:     85,293 rows


## 2. Clean Players

In [5]:
# EDA: 49 players missing date_of_birth — drop (age is an essential feature)
players = players.dropna(subset=['date_of_birth'])
print(f'After dropping missing DOB: {len(players):,} players')

# Age computed at 2025 for consistency across the entire dataset
players['age'] = 2025 - players['date_of_birth'].dt.year
print(f'Age range before filter: {players["age"].min()} – {players["age"].max()} years')

# EDA: age range is 15–57; players > 45 are almost certainly retired with stale/outdated data
players = players[(players['age'] >= 15) & (players['age'] <= 45)].copy()
print(f'After filtering age 15–45: {len(players):,} players')

After dropping missing DOB: 33,703 players
Age range before filter: 15 – 45 years
After filtering age 15–45: 33,703 players


In [7]:
# EDA: height_in_cm contains values < 100 cm — data entry errors, set to NaN
bad_height = players['height_in_cm'] < 100
print(f'Erroneous height values (< 100 cm): {bad_height.sum()}')
players.loc[bad_height, 'height_in_cm'] = np.nan

# Impute remaining missing heights (~7%) with the column median
median_height = players['height_in_cm'].median()
players['height_in_cm'] = players['height_in_cm'].fillna(median_height)
print(f'Height median used for imputation: {median_height:.1f} cm')

Erroneous height values (< 100 cm): 0
Height median used for imputation: 183.0 cm


In [8]:
# EDA: foot missing for ~7% of players — fill with 'unknown'
players['foot'] = players['foot'].fillna('unknown')
print('Foot distribution after imputation:')
print(players['foot'].value_counts().to_string())

Foot distribution after imputation:
foot
right      21865
left        7851
unknown     2585
both        1402


In [9]:
# EDA: 189 players have position = 'Missing' — no position group can be assigned, drop them
print(f"Position distribution before drop:\n{players['position'].value_counts().to_string()}\n")

players = players[players['position'] != 'Missing'].copy()
print(f'After dropping Missing position: {len(players):,} players')

# Map to 4 position groups used for separate model training
pos_map = {'Goalkeeper': 'GK', 'Defender': 'DEF', 'Midfield': 'MID', 'Attack': 'ATT'}
players['position_group'] = players['position'].map(pos_map)
print(f"\nPosition group counts:\n{players['position_group'].value_counts().to_string()}")

Position distribution before drop:
position
Defender      10714
Midfield       9766
Attack         9299
Goalkeeper     3745
Missing         179

After dropping Missing position: 33,524 players

Position group counts:
position_group
DEF    10714
MID     9766
ATT     9299
GK      3745


In [10]:
# highest_market_value_in_eur: only ~8% missing (same as market_value_in_eur per EDA)
# Captures peak career value — different signal from current value, worth keeping as feature
players['highest_market_value_in_eur'] = players['highest_market_value_in_eur'].fillna(0)

# Retain only modelling-relevant columns.
# Dropped: agent_name (48% missing), contract_expiration_date (36% missing),
#          image_url, url, player_code, country_of_birth, city_of_birth,
#          country_of_citizenship, sub_position, first_name, last_name, etc.
keep = ['player_id', 'name', 'position', 'position_group', 'age', 'height_in_cm', 'foot',
        'highest_market_value_in_eur']
players_clean = players[keep].copy()

print(f'Cleaned players shape: {players_clean.shape}')
print(players_clean.dtypes)

Cleaned players shape: (33524, 8)
player_id                        int64
name                               str
position                           str
position_group                     str
age                              int32
height_in_cm                   float64
foot                               str
highest_market_value_in_eur    float64
dtype: object


## 3. Aggregate Appearances

In [11]:
# Sum all career appearances per player (no date filtering — full career stats)
# Column names match notebook 02 EDA conventions
agg = appearances.groupby('player_id').agg(
    total_appearances=('appearance_id', 'count'),
    total_goals      =('goals',          'sum'),
    total_assists    =('assists',         'sum'),
    total_minutes    =('minutes_played',  'sum'),
    total_yellow     =('yellow_cards',    'sum'),
    total_red        =('red_cards',       'sum'),
).reset_index()

print(f'Players with appearance records: {len(agg):,}')
print('\n99th percentile values (used for capping in next cell):')
for col in ['total_goals', 'total_assists', 'total_minutes', 'total_appearances']:
    print(f'  {col:<22}: {agg[col].quantile(0.99):.0f}')

# EDA: total_red has median=0 for the vast majority of players — very low signal
# Kept for completeness; likely dropped during feature selection in notebook 05
print(f'\nNote — total_red: median={agg["total_red"].median():.0f},  max={agg["total_red"].max():.0f}  (near-zero for most players)')

Players with appearance records: 26,489

99th percentile values (used for capping in next cell):
  total_goals           : 78
  total_assists         : 53
  total_minutes         : 28528
  total_appearances     : 376

Note — total_red: median=0,  max=7  (near-zero for most players)


In [12]:
# EDA outlier figure: extreme values confirmed (goals max ~480, assists ~250, minutes ~48,000)
# Cap at 99th percentile HERE so all downstream processed CSVs are already clean
print('Capping performance stats at 99th percentile:')
cap_cols = ['total_goals', 'total_assists', 'total_minutes', 'total_appearances', 'total_yellow']
for col in cap_cols:
    cap = agg[col].quantile(0.99)
    n_capped = (agg[col] > cap).sum()
    agg[col] = agg[col].clip(upper=cap)
    print(f'  {col:<22}: cap={cap:.0f}   ({n_capped} players affected)')

Capping performance stats at 99th percentile:
  total_goals           : cap=78   (260 players affected)
  total_assists         : cap=53   (261 players affected)
  total_minutes         : cap=28528   (265 players affected)
  total_appearances     : cap=376   (265 players affected)
  total_yellow          : cap=68   (263 players affected)


## 4. Clean Valuations

In [13]:
# Use only the most recent valuation per player
# EDA showed the dataset spans 2010–2024; latest entry is most representative of current value
latest_vals = (
    valuations
    .dropna(subset=['date'])
    .sort_values('date')
    .groupby('player_id')
    .last()
    .reset_index()
)

print(f'Players with a valuation: {len(latest_vals):,}')
print(f'Date range: {latest_vals["date"].min().date()} → {latest_vals["date"].max().date()}')

Players with a valuation: 31,375
Date range: 2005-12-02 → 2026-02-20


In [14]:
# Add convenience columns:
#   market_value_m   — value in €M for human-readable summaries
#   log_market_value — the actual model target (EDA: raw distribution skewness >> 2)
#   valuation_year   — lets notebook 05 filter by recency if needed
#                      (EDA: 2024 mean spiked to €3.7M due to top-player inflation)
latest_vals['market_value_m']   = latest_vals['market_value_in_eur'] / 1_000_000
latest_vals['log_market_value'] = np.log1p(latest_vals['market_value_in_eur'])
latest_vals['valuation_year']   = latest_vals['date'].dt.year

val_clean = latest_vals[['player_id', 'market_value_in_eur', 'market_value_m',
                          'log_market_value', 'valuation_year']].copy()

mv  = val_clean['market_value_m'].dropna()
lmv = val_clean['log_market_value'].dropna()
print(f'market_value_m   — median: €{mv.median():.2f}M   mean: €{mv.mean():.2f}M   max: €{mv.max():.1f}M')
print(f'log_market_value — median: {lmv.median():.2f}   mean: {lmv.mean():.2f}   std: {lmv.std():.2f}')
print(f'valuation_year   — range: {int(val_clean["valuation_year"].min())} – {int(val_clean["valuation_year"].max())}')

market_value_m   — median: €0.25M   mean: €1.66M   max: €200.0M
log_market_value — median: 12.43   mean: 12.65   std: 1.57
valuation_year   — range: 2005 – 2026


## 5. Clean Sheets  (via club_games + game_lineups)

In [15]:
# Clean sheet = game where the team conceded 0 goals
# Total goals conceded = opponent_goals + own_goals (Transfermarkt schema)
cs_games = club_games[
    (club_games['opponent_goals'] == 0) & (club_games['own_goals'] == 0)
][['game_id', 'club_id']].drop_duplicates()

print(f'Clean sheet game-club entries: {len(cs_games):,}')

# Find which players appeared in those games for those clubs
cs_players = lineups.merge(cs_games, on=['game_id', 'club_id'], how='inner')

# Count clean sheets per player
clean_sheets = cs_players.groupby('player_id').size().reset_index(name='clean_sheets')

print(f'Players with ≥1 clean sheet: {len(clean_sheets):,}')
print(f'Max clean sheets per player:  {clean_sheets["clean_sheets"].max()}')

Clean sheet game-club entries: 9,600
Players with ≥1 clean sheet: 31,175
Max clean sheets per player:  46


## 6. Club Features  (via clubs.csv)

In [16]:
# players.csv already has current_club_domestic_competition_id (the player's current league)
# clubs.csv adds squad_size for that club — a proxy for club size/ambition
# Join: players.current_club_id → clubs.club_id

squad_lookup = clubs[['club_id', 'squad_size']].rename(columns={'club_id': 'current_club_id'})

club_info = players[['player_id', 'current_club_id', 'current_club_domestic_competition_id']].copy()
club_info = club_info.merge(squad_lookup, on='current_club_id', how='left')

# Fill missing squad_size with median (some clubs not in clubs.csv)
club_info['squad_size'] = club_info['squad_size'].fillna(club_info['squad_size'].median())
club_info['current_club_domestic_competition_id'] = (
    club_info['current_club_domestic_competition_id'].fillna('unknown')
)

club_features = club_info[['player_id', 'current_club_domestic_competition_id', 'squad_size']].copy()

print(f'Club features shape: {club_features.shape}')
print(f'Unique leagues: {club_features["current_club_domestic_competition_id"].nunique()}')
print(f'Squad size — median: {club_features["squad_size"].median():.0f},  max: {club_features["squad_size"].max():.0f}')

Club features shape: (33524, 3)
Unique leagues: 14
Squad size — median: 27,  max: 41


## 7. Transfer Features  (via transfers.csv)

In [17]:
# Aggregate transfer history per player:
#   transfer_count  — number of transfers (proxy for experience / market activity)
#   total_fees_eur  — total fees received across career (capped at 99th pct)
# Note: transfer_fee=0 means free transfer (not missing); NaN means unknown
transfers['transfer_fee'] = pd.to_numeric(transfers['transfer_fee'], errors='coerce')

trans_agg = transfers.groupby('player_id').agg(
    transfer_count =('player_id',    'count'),
    total_fees_eur =('transfer_fee', 'sum'),
).reset_index()

# Cap total fees at 99th percentile (extreme outliers: e.g. Neymar €222M)
fee_cap = trans_agg['total_fees_eur'].quantile(0.99)
trans_agg['total_fees_eur'] = trans_agg['total_fees_eur'].clip(upper=fee_cap).fillna(0)

print(f'Players with transfer records: {len(trans_agg):,}')
print(f'Transfer count — median: {trans_agg["transfer_count"].median():.0f},  max: {trans_agg["transfer_count"].max()}')
print(f'Total fees cap (99th pct):  €{fee_cap / 1e6:.1f}M')

Players with transfer records: 11,033
Transfer count — median: 7,  max: 35
Total fees cap (99th pct):  €81.4M


## 8. Merge All Datasets

In [18]:
# All LEFT JOINs on player_id — preserve all cleaned players
df = players_clean.copy()
print(f'Start (cleaned players):     {len(df):,}')

df = df.merge(agg,           on='player_id', how='left')
print(f'After +appearances:          {len(df):,}')

df = df.merge(val_clean,     on='player_id', how='left')
print(f'After +valuations:           {len(df):,}')

df = df.merge(clean_sheets,  on='player_id', how='left')
print(f'After +clean_sheets:         {len(df):,}')

df = df.merge(club_features, on='player_id', how='left')
print(f'After +club_features:        {len(df):,}')

df = df.merge(trans_agg,     on='player_id', how='left')
print(f'After +transfer_features:    {len(df):,}')

Start (cleaned players):     33,524
After +appearances:          33,524
After +valuations:           33,524
After +clean_sheets:         33,524
After +club_features:        33,524
After +transfer_features:    33,524


In [19]:
# Fill missing performance stats with 0 (player has no recorded appearances)
perf_cols = ['total_appearances', 'total_goals', 'total_assists',
             'total_minutes', 'total_yellow', 'total_red', 'clean_sheets']
for col in perf_cols:
    df[col] = df[col].fillna(0)

# EDA: median appearances = 30; flag players with 0 recorded appearances
# Distinguishes new/youth players from missing data — useful signal for the model
df['has_appearances'] = (df['total_appearances'] > 0).astype(int)
print(f'Players with ≥1 appearance:  {(df["has_appearances"] == 1).sum():,}')
print(f'Players with 0 appearances:  {(df["has_appearances"] == 0).sum():,}  (has_appearances=0)')

# Transfer features: 0 for players with no transfer history
df['transfer_count']  = df['transfer_count'].fillna(0)
df['total_fees_eur']  = df['total_fees_eur'].fillna(0)

# Club features: fill missing league with 'unknown', squad_size with median
df['current_club_domestic_competition_id'] = df['current_club_domestic_competition_id'].fillna('unknown')
df['squad_size'] = df['squad_size'].fillna(df['squad_size'].median())

# market_value columns stay as NaN for players with no recorded valuation
# (kept in dataset; excluded from model training via dropna in notebook 05)
n_no_val = df['market_value_in_eur'].isna().sum()
print(f'\nPlayers with valuation:    {len(df) - n_no_val:,}')
print(f'Players without valuation: {n_no_val:,}  (NaN — excluded during modelling)')
print(f'\nFinal shape: {df.shape}')
print(f'Columns: {df.columns.tolist()}')

Players with ≥1 appearance:  25,886
Players with 0 appearances:  7,638  (has_appearances=0)

Players with valuation:    30,710
Players without valuation: 2,814  (NaN — excluded during modelling)

Final shape: (33524, 24)
Columns: ['player_id', 'name', 'position', 'position_group', 'age', 'height_in_cm', 'foot', 'highest_market_value_in_eur', 'total_appearances', 'total_goals', 'total_assists', 'total_minutes', 'total_yellow', 'total_red', 'market_value_in_eur', 'market_value_m', 'log_market_value', 'valuation_year', 'clean_sheets', 'current_club_domestic_competition_id', 'squad_size', 'transfer_count', 'total_fees_eur', 'has_appearances']


## 9. Split by Position Group

In [20]:
df_gk  = df[df['position_group'] == 'GK'].copy()
df_def = df[df['position_group'] == 'DEF'].copy()
df_mid = df[df['position_group'] == 'MID'].copy()
df_att = df[df['position_group'] == 'ATT'].copy()

print('Position group sizes:')
print(f'  GK:  {len(df_gk):,}')
print(f'  DEF: {len(df_def):,}')
print(f'  MID: {len(df_mid):,}')
print(f'  ATT: {len(df_att):,}')

Position group sizes:
  GK:  3,745
  DEF: 10,714
  MID: 9,766
  ATT: 9,299


## 10. Summary

In [21]:
rows = []
for label, g in [('GK', df_gk), ('DEF', df_def), ('MID', df_mid), ('ATT', df_att)]:
    with_val = g.dropna(subset=['market_value_in_eur'])
    rows.append({
        'Group':            label,
        'Total':            len(g),
        'With valuation':   len(with_val),
        'Avg age':          round(g['age'].mean(), 1),
        'Median MV (€M)':  round(with_val['market_value_m'].median(), 2),
        'Mean MV (€M)':    round(with_val['market_value_m'].mean(),   2),
        'Avg clean sheets': round(g['clean_sheets'].mean(), 1),
        'Avg appearances':  round(g['total_appearances'].mean(), 1),
    })

summary = pd.DataFrame(rows).set_index('Group')
print(summary.to_string())

       Total  With valuation  Avg age  Median MV (€M)  Mean MV (€M)  Avg clean sheets  Avg appearances
Group                                                                                                 
GK      3745            3313     29.4            0.15          0.79               4.5             30.7
DEF    10714            9867     29.9            0.25          1.57               4.8             53.1
MID     9766            8976     29.6            0.25          1.85               4.6             53.8
ATT     9299            8554     29.4            0.30          2.02               4.2             51.1


## 11. Validation

In [24]:
errors = []

# 1. No duplicate player_ids
dupes = df.duplicated('player_id').sum()
if dupes > 0: errors.append(f"duplicate player_ids: {dupes}")
else: print(f"✓  No duplicate player_ids")

# 2. Age range
if df['age'].min() < 15 or df['age'].max() > 45:
    errors.append(f"age out of range: [{df['age'].min()}, {df['age'].max()}]")
else: print(f"✓  Age range: {df['age'].min()}–{df['age'].max()}")

# 3. No 'Missing' positions
if (df['position'] == 'Missing').any():
    errors.append("'Missing' positions still present")
else: print(f"✓  No 'Missing' positions")

# 4. 99th pct caps applied (values from EDA)
caps = {'total_goals': 78, 'total_assists': 53, 'total_minutes': 28528, 'total_appearances': 376}
for col, cap in caps.items():
    actual = df[col].max()
    if actual > cap: errors.append(f"{col} not capped: max={actual:.0f} > {cap}")
    else: print(f"✓  {col} capped (max={actual:.0f})")

# 5. No NaN in columns that should be filled
must_fill = ['total_appearances', 'total_goals', 'total_assists', 'total_minutes',
             'total_yellow', 'total_red', 'clean_sheets', 'has_appearances',
             'transfer_count', 'total_fees_eur', 'squad_size',
             'current_club_domestic_competition_id']
for col in must_fill:
    n = df[col].isna().sum()
    if n > 0: errors.append(f"{col} has {n} NaN")
    else: print(f"✓  {col} — no NaN")

# 6. All 4 position groups present
for pos in ['GK', 'DEF', 'MID', 'ATT']:
    n = (df['position_group'] == pos).sum()
    if n == 0: errors.append(f"position group {pos} is empty")
    else: print(f"✓  {pos}: {n:,} players")

# 7. log_market_value is monotonic with market_value_in_eur
sample = df.dropna(subset=['market_value_in_eur']).head(1000)
if not (sample['log_market_value'] >= 0).all():
    errors.append("log_market_value has negative values")
else: print(f"✓  log_market_value non-negative")

print(f"\n{'='*45}")
if errors:
    print(f"FAILED — {len(errors)} issue(s):")
    for e in errors: print(f"  ✗  {e}")
else:
    print(f"ALL {len(must_fill) + len(caps) + 4} CHECKS PASSED ✓")
    print(f"  Final dataset: {df.shape[0]:,} players × {df.shape[1]} columns")
print('='*45)

✓  No duplicate player_ids
✓  Age range: 15–45
✓  No 'Missing' positions
✓  total_goals capped (max=78)
✓  total_assists capped (max=53)
✓  total_appearances — no NaN
✓  total_goals — no NaN
✓  total_assists — no NaN
✓  total_minutes — no NaN
✓  total_yellow — no NaN
✓  total_red — no NaN
✓  clean_sheets — no NaN
✓  has_appearances — no NaN
✓  transfer_count — no NaN
✓  total_fees_eur — no NaN
✓  squad_size — no NaN
✓  current_club_domestic_competition_id — no NaN
✓  GK: 3,745 players
✓  DEF: 10,714 players
✓  MID: 9,766 players
✓  ATT: 9,299 players
✓  log_market_value non-negative

FAILED — 2 issue(s):
  ✗  total_minutes not capped: max=28528 > 28528
  ✗  total_appearances not capped: max=376 > 376


## 12. Save Outputs

In [ ]:
df.to_csv(    MERGED_PATH + 'merged_complete.csv', index=False)
df_gk.to_csv( MERGED_PATH + 'merged_gk.csv',      index=False)
df_def.to_csv(MERGED_PATH + 'merged_def.csv',      index=False)
df_mid.to_csv(MERGED_PATH + 'merged_mid.csv',      index=False)
df_att.to_csv(MERGED_PATH + 'merged_att.csv',      index=False)

print(f'Saved to {os.path.abspath(MERGED_PATH)}')
for fname in ['merged_complete.csv', 'merged_gk.csv', 'merged_def.csv', 'merged_mid.csv', 'merged_att.csv']:
    size = os.path.getsize(MERGED_PATH + fname) / 1024
    print(f'  {fname:<25} {size:>7.1f} KB')